# 실제 분석데이터용으로 사용할 데이터셋 만들어보기
*결측치처리 + 중복값처리 + 패딩완료한 csv 파일
*resize, 정규화 진해한 이미지파일

In [17]:
import dask.dataframe as dd
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2

csv파일 불러오기

In [2]:
# df = dd.read_csv('sampled_data/*/*.csv')
df = dd.read_csv('sampled_data/*/*.csv',dtype={'외부 누적일사 평균': 'object'})

중복값 처리

In [15]:
df.compute()

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 온도 2 평균,내부 온도 2 최고,내부 온도 2 최저,내부 온도 3 평균,내부 온도 3 최고,내부 온도 3 최저,...,배지 중량 최저,양액 온도 평균,양액 온도 최고,양액 온도 최저,외부 풍향 수치,외부 풍향,외부 빗물 시간,외부 누적일사 평균,양액 급액 누적,양액 배액 누적
0,2021-07-14 10:10:00,31.7,32.0,31.3,-,-,-,-,-,-,...,-,-,-,-,-,-,-,302.6,-,-
1,2021-07-14 10:00:00,31.2,31.3,30.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,260.8,-,-
2,2021-07-14 09:50:00,31.4,31.4,31.4,-,-,-,-,-,-,...,-,-,-,-,-,-,-,196.3,-,-
3,2021-07-14 09:40:00,31.3,31.4,31.1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,333.5,-,-
4,2021-07-14 09:30:00,30.7,31.0,30.5,-,-,-,-,-,-,...,-,-,-,-,-,-,-,663.5,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,2021-09-12 09:50:00,25.2,25.3,25.1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
290,2021-09-12 09:40:00,25.1,25.3,24.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
291,2021-09-12 09:30:00,25.4,25.5,25.4,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
292,2021-09-12 09:20:00,25.5,25.7,25.3,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [16]:
df.drop_duplicates(subset=['측정시각'])

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 온도 2 평균,내부 온도 2 최고,내부 온도 2 최저,내부 온도 3 평균,내부 온도 3 최고,내부 온도 3 최저,내부 온도 4 평균,내부 온도 4 최고,내부 온도 4 최저,내부 습도 1 평균,내부 습도 1 최고,내부 습도 1 최저,내부 습도 2 평균,내부 습도 2 최고,내부 습도 2 최저,내부 습도 3 평균,내부 습도 3 최고,내부 습도 3 최저,내부 습도 4 평균,내부 습도 4 최고,내부 습도 4 최저,내부 이슬점 평균,내부 이슬점 최고,내부 이슬점 최저,내부 CO2 평균,내부 CO2 최고,내부 CO2 최저,외부 풍속 평균,외부 풍속 최고,외부 풍속 최저,내부 EC 1 평균,내부 EC 1 최고,내부 EC 1 최저,내부 PH 1 평균,내부 PH 1 최고,내부 PH 1 최저,배지 중량 평균,배지 중량 최고,배지 중량 최저,양액 온도 평균,양액 온도 최고,양액 온도 최저,외부 풍향 수치,외부 풍향,외부 빗물 시간,외부 누적일사 평균,양액 급액 누적,양액 배액 누적
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,float64,float64,float64,object,object,object,object,object,object,object,object,object,float64,float64,float64,object,object,object,object,object,object,object,object,object,float64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [17]:
df.drop_duplicates(subset=['측정시각']).compute()

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 온도 2 평균,내부 온도 2 최고,내부 온도 2 최저,내부 온도 3 평균,내부 온도 3 최고,내부 온도 3 최저,...,배지 중량 최저,양액 온도 평균,양액 온도 최고,양액 온도 최저,외부 풍향 수치,외부 풍향,외부 빗물 시간,외부 누적일사 평균,양액 급액 누적,양액 배액 누적
0,2021-07-14 10:10:00,31.7,32.0,31.3,-,-,-,-,-,-,...,-,-,-,-,-,-,-,302.6,-,-
1,2021-07-14 10:00:00,31.2,31.3,30.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,260.8,-,-
2,2021-07-14 09:50:00,31.4,31.4,31.4,-,-,-,-,-,-,...,-,-,-,-,-,-,-,196.3,-,-
3,2021-07-14 09:40:00,31.3,31.4,31.1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,333.5,-,-
4,2021-07-14 09:30:00,30.7,31.0,30.5,-,-,-,-,-,-,...,-,-,-,-,-,-,-,663.5,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,2021-07-11 09:10:00,35.2,35.5,35.0,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
584,2021-07-11 09:00:00,34.4,35.0,33.8,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
586,2021-07-11 08:50:00,33.3,33.7,32.8,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
292,2021-09-12 09:20:00,25.5,25.7,25.3,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


간단하게 아래처럼 처리하면 csv간 합쳐진 상태로 중복값이 사라져서 이상하게 되는듯함, csv간 겹치는 구간이 존재할것이기 때문

In [18]:
df_drop_duplicated = df.drop_duplicates(subset=['측정시각'])

In [22]:
partitions = df_drop_duplicated.to_delayed()
shapes = [part.values.shape for part in partitions]
results = dask.compute(*shapes)
results

((2694, 52),)

In [24]:
partitions = df.to_delayed()
shapes = [part.values.shape for part in partitions]
results = dask.compute(*shapes)
results

((291, 52),
 (588, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (588, 52),
 (588, 52),
 (588, 52),
 (291, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (588, 52),
 (294, 52),
 (294, 52),
 (588, 52),
 (588, 52),
 (294, 52))

#각 partition별로 처리해보자

*각 파티션은 csv를 불러온 데이터 프레임을 의미하고 여기에 values를 적용하면 어레이값으로 출력되는것임

*판다스 데이터프레임과 동일

In [28]:
#############성공##############
partitions = df.to_delayed()
shapes = [part.drop_duplicates(subset=['측정시각']).values.shape for part in partitions]
results = dask.compute(*shapes)
results

((291, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (291, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52))

csv 데이터 만들기

In [19]:
# df = dd.read_csv('sampled_data/*/*.csv')
df = dd.read_csv('sampled_data/*/*.csv',dtype={'외부 누적일사 평균': 'object'})

csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

In [73]:
#############성공##############
@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    pad[:length] = data[:length]
    return pad

partitions = df.to_delayed()
datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
# results = dask.compute(*datas)
# results
datas

[Delayed('padding-aa4ab051-df2f-4efe-9888-be6ed42ef509'),
 Delayed('padding-8f74c97f-e693-4bad-b844-50a3035b423c'),
 Delayed('padding-9e144c01-55b7-4885-96de-c807fd266403'),
 Delayed('padding-8eaea867-4167-4408-8211-a2f94a1ced18'),
 Delayed('padding-6b89bcb5-c00f-452e-8933-2fc791841755'),
 Delayed('padding-dd21679d-18fa-416b-a8fe-f5d1eca39f06'),
 Delayed('padding-f2633ebc-0589-446b-ae98-a9458cb5cbe0'),
 Delayed('padding-6a0f458f-fef9-40b9-9657-df6706b8aab4'),
 Delayed('padding-a2675632-2256-4299-a728-878afd748b60'),
 Delayed('padding-d0811065-6c28-4578-a80c-80218aa1d196'),
 Delayed('padding-c2eb9bdf-1b8d-4102-af03-7ec4bd06c07f'),
 Delayed('padding-dadeb3d3-4993-4996-9165-3e28a6456448'),
 Delayed('padding-82c51ed3-b555-4f5a-bb81-1c0b13c31839'),
 Delayed('padding-75e1021d-7f95-43c3-b860-dd9c0b326f40'),
 Delayed('padding-13ae18f1-f9a2-4017-be78-a3e7ecb72247'),
 Delayed('padding-73a9d9bf-0580-47b2-89e9-f09007082337'),
 Delayed('padding-7198e65d-2804-4374-be90-5bb849571a5e'),
 Delayed('padd

# 이미지데이터셋 만들기

In [3]:
# df = dd.read_csv('sampled_data/*/*.csv')
imageset = dask_image.imread.imread('sampled_data/*/*.jpg')

In [41]:
@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img
# partitions = imageset.to_delayed()
datas = [img_resize(part) for part in imageset]
results = dask.compute(*datas)
np.array(results).shape
# # results
# datas
# results

(21, 224, 224, 3)

TypeError: 'module' object is not callable